## Goals:
    1. Take in DF's from 'filtered' (using pandas)
    2. Read first row, get first item in species column
    3. Use species[1] to look up sequence in fasta file
    4. User first row, raw position # to get nucleotide position + n (length of motif)
    5. Put this nucleotide sequence into new column

TO-DO:
1. read in motif file and get motif length
2. make alignment file method for raw file
3. when reading file, need to account for negative/positive
4. make alignment file method --> need to account for spaces???

Account for negative string reading GGATTA
                                    CCTAAT
                                    
-bcd:
- GGATTA positive
- TAATCC negative

In [1]:
from Bio import SeqIO
from Bio import Seq
import pandas as pd
import numpy as np
import csv
import re
import os
from datascience import *

In [13]:
def seqToString(motif):
    """
    motif: a Seq that represents the motif 
    
    Returns the String representation of the motif
    """
    i = 0
    string = ""
    length = motif.__len__()
    while i < length:
        string += motif.__getitem__(i)
        i += 1
    return string

In [14]:
def getMotifLength(motif_key):
    f = open("../data/jaspar_fm/modified/" + motif_key, 'r')
    length = 0
    for line in f:
        a = line.split()
        length = len(a)
    return length

In [15]:
getMotifLength("MA0049.1_hb.jaspar")

10

In [16]:
def getNegative(pos_seq):
    """
    pos_seq = dna sequence in the positive direction reading from the file
    
    Returns the negative counterpart of the positive sequence.
    """
    dict = {"A":'T','T':'A','G':'C','C':'G','-':'-'}
    negative = ""
    last_index = len(pos_seq) - 1
    while last_index > -1:
        negative += dict[pos_seq[last_index].upper()]
        last_index -= 1
    return negative

In [17]:
def getRawMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Creates a reader representing the data frame and iterates through each line retrieving the species ID and key.
    Parses through the raw file to find the motif sequence.
    Returns an data frame of the motifs.
    """
    f = open("../data/output/bcd/" + csv_file, 'r')
    data = csv.reader(f)
    line0 = next(data)
    motifs = []
    for line in data:
        species = line[2]
        direction = line[4]
        raw_index = int(float(line[3]))
        key = species.split('|')[0]
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_" + key + ".fa", "fasta"))
        sequence = record_dict[species]
        motif_key = line[6]
        motif_length = getMotifLength(motif_key);
        motif = sequence[raw_index:raw_index + motif_length]
        string_motif = seqToString(motif)
        if direction == "negative":
            string_motif = getNegative(string_motif)
        motifs += [string_motif]
    f.close()
    motif_df = pd.DataFrame({'motif sequences': motifs})
    return motifs

In [18]:
def getAlignMotifStrings(csv_file):
    f = open("../data/output/bcd/" + csv_file, 'r')
    data = csv.reader(f)
    line0 = next(data)
    motifs = []
    for line in data:
        species = line[2]
        direction = line[4]
        align_index = int(float(line[5]))
        key = species.split('|')[0]
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/alignments/align_outlier_rm_with_length_" + key + ".fa", "fasta"))
        sequence = record_dict[species]
        motif_key = line[6]
        motif_length = getMotifLength(motif_key);
        motif = sequence[align_index:align_index + motif_length]
        string_motif = seqToString(motif)
        if direction == "negative":
            string_motif = getNegative(string_motif)
        motifs += [string_motif]
    f.close()
    motif_df = pd.DataFrame({'motif sequences': motifs})
    return motifs

In [19]:
motif = getAlignMotifStrings("occurance_align_outlier_rm_with_length_VT6436.fa.csv")
df = pd.DataFrame()
df["VT6436.fa.csv"] = motif
# print(df)
motif2 = getAlignMotifStrings("occurance_align_outlier_rm_with_length_VT6705.fa.csv")
df1 = pd.DataFrame()
df1["VT6705.fa.csv"] = motif2
# print(df1)
df = pd.concat([df,df1], ignore_index=False, axis = 1)
#print(df)

FileNotFoundError: [Errno 2] No such file or directory: '../data/output/bcd/occurance_align_outlier_rm_with_length_VT6436.fa.csv'

In [26]:
df = pd.DataFrame()
if df.empty:
    print("empty")

empty


In [27]:
def alignmentsDataFrame():
    csvList = os.listdir("../data/output/bcd/")
    df = pd.DataFrame()
    for file in csvList:
        motif = getAlignMotifStrings(file)
        df1 = pd.DataFrame()
        df1[file[39:]] = motif
        if df.empty:
            df[file[39:]] = motif
        else:
            df = pd.concat([df,df1], ignore_index=False, axis=1)
    new_csv = df.to_csv("../data/output/2.motif_extraction_output/align/bcd_motif.fa.csv")
    print(df)

In [28]:
alignmentsDataFrame()

    VT11048.fa.csv VT16679.fa.csv VT19895.fa.csv VT40548.fa.csv  \
0           TAATCC         TTATCC         TAATCC         TAATCC   
1           TAATCC         -----C         TT-ATC         TTATCC   
2              NaN         TAATCC         TAATCC         TTATCC   
3              NaN         TTATCC         TT-ATC         TAATCC   
4              NaN         T-----         TAATCC         TTATCC   
5              NaN         -----C         TAATCC         TAATCC   
6              NaN         TAATCC         TAATCC         TAATCC   
7              NaN         TTATCC         TAATCC         TTATCC   
8              NaN         TTATCC         A--TCC         TAATCC   
9              NaN         TTATCC         TTATCC         TAATCC   
10             NaN         -----C         TAATCC         TAATCC   
11             NaN         TAATCC         A--TCC         TAATCC   
12             NaN         TTATCC         TAATCC         TTA---   
13             NaN         TTATCC         TAATCC         TAATC

In [29]:
def csvToDataFrame(csv_file):
    df = pd.read_csv("../data/output/bcd/" + csv_file);
    return df

In [30]:
def csvToArray(csv_file):
    f = open(csv_file, 'r')
    data = csv.reader(f)
    labels = next(data)
    labels.append('motif sequence')
    array = []
    array.append(labels)
    
    for row in data:
        array.append(row)
    return array

In [35]:
def addRawMotifStrings(csv_file):
    """
    csv_file: a csv file that contains the 'filtered' data frame
    
    Appends the DF of motif sequences as a column onto the filtered DF.
    """
    raw_motif_seq = getRawMotifStrings(csv_file)
    df = csvToDataFrame(csv_file)
    df['raw motif sequences'] = raw_motif_seq
    new_csv = df.to_csv("../data/output/2.motif_extraction_output/raw/motif+" + csv_file)
    return df

In [36]:
d = addRawMotifStrings("occurance_align_outlier_rm_with_length_VT11048.fa.csv")

In [37]:
d

,Unnamed: 0,score,species,raw_position,strand,align_position,motif,raw motif sequences
0,0,11.612828,VT11048|0|MEMB005D|-|287,39,negative,60,MA0212.1_bcd.jaspar,TAATCC
1,1,11.612828,VT11048|0|MEMB006A|+|286,38,negative,60,MA0212.1_bcd.jaspar,TAATCC


In [179]:
csvList = os.listdir("../data/output/bcd/")
for file in csvList:
    addRawMotifStrings(file)

In [5]:
species = [11048, 16679, 19895, 40548, 43692, 44110, 48156, 50550, 59000, 60074, 61209, 6436, 6705, 7859, 8646]
member = ['Dkik', 'MEMB002A', 'MEMB002B', 'MEMB002C', 'MEMB002D', 'MEMB002E', 'MEMB002F', 'MEMB003A', 'MEMB003B', 'MEMB003C', 'MEMB003D',
          'MEMB003E', 'MEMB003F', 'MEMB004A', 'MEMB004B', 'MEMB004E', 'MEMB005D', 'MEMB006B', 'MEMB006C', 'MEMB007A', 'MEMB007B', 
          'MEMB007C', 'MEMB007D', 'MEMB008C']

In [6]:
def getAlignPositions(species):
    table = Table.read_table("../data/output/map_motif_bcd_with_threshold/occurance_align_outlier_rm_with_length_VT" + str(species) + ".fa.csv")
    align = table.group('align_position').drop('count').relabel('align_position', 'align_' +  str(species)).column('align_' +  str(species))
    return align   


In [7]:
a = getAlignPositions(11048)
a

array([60])

In [8]:
spec_align_dict = {}
for s in species:
    a = getAlignPositions(s)
    spec_align_dict[s] = a
print(spec_align_dict)

{11048: array([60]), 16679: array([ 502, 1200, 1202, 1316, 1481, 1716, 1717, 2073, 2114, 2202, 2606,
       2612, 2618]), 19895: array([  93,  518,  611, 2385, 2444, 3390]), 40548: array([ 619,  672,  764,  867,  960, 1077, 1278, 1736, 1905, 1963, 1967,
       2020, 2649, 2703, 3011, 3028, 3234, 3243, 3244, 3269, 3306, 3307,
       3318, 3327, 3465, 3493, 3902]), 43692: array([ 600,  677,  685,  744, 1308, 1750, 1813, 2554, 2622, 2915, 3098,
       3295]), 44110: array([  41,  581,  696,  715,  753,  854,  995, 1013, 1049, 1716, 1745,
       1943, 1955, 2071]), 48156: array([153]), 50550: array([ 325,  345,  527,  589,  617,  906, 1092, 1111, 1114, 1124, 1152,
       1158, 1183, 1593, 1992, 2923, 3132, 3333]), 59000: array([ 457,  647,  934, 1881, 3177, 3330, 3339, 3806, 3841, 3975, 4043,
       4739, 5411, 5432, 5468, 5545, 5626, 5991, 6378]), 60074: array([ 44, 575, 866, 940]), 61209: array([ 579, 1064, 1077, 2099, 2951, 3341, 3360, 3575, 3680, 3689, 5942,
       6074, 7345, 7628]), 

swap if negative leave alone if positive
Rather than from the original massive file, i need to be pulling from: motif+occurance_align_outlier_rm_with_length_VT19895.fa
       - the strand direction
       - the score
so maybe merge the two tables first based on raw position?

In [28]:
def getRawPositions(species, length):
    table = Table.read_table("../data/output/map_motif_bcd_no_threshold/VT" + str(species) + ".fa.csv")
    spec = Table()
    raw_position = []
    a = spec_align_dict[species]
    print(a)
    for pos in a:
        print(table)
        raw = table.where('align_position', are.equal_to(pos)).where('strand', are.equal_to('positive'))
        raw = raw.drop('Unnamed: 0', 'motif').relabel('raw_position', str(pos))
        arr = raw.column(str(pos))
        spec = Table().with_columns('raw', arr, 'orig_species', raw.column('species'), 'score', raw.column('score'), 'align_position', raw.column('align_position'), 'strand', raw.column('strand'))
        spec.show()
        record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_VT" + str(species) + ".fa", "fasta"))
        species_dict = dict(zip(spec.column(1), spec.column(0)))
        seq = []
        before = []
        after = []
        all_pos = Table()
        raw_arr = spec.column('raw')
        species_arr = spec.column('orig_species')
        score_arr = spec.column('score')
        align_arr = spec.column('align_position')
        strand_arr = spec.column('strand')
        for key in species_dict: 
            #get the index of the species in its array and get that value from the strand array and then check pos or neg. swap if negative leave if pos
            sequence = record_dict[key]
            seq.append(getNegative(seqToString(sequence[species_dict[key]:species_dict[key] + 6])))
            before.append(getNegative(seqToString(sequence[species_dict[key] - 6 - length:species_dict[key] - 6])))
            after.append(getNegative(seqToString(sequence[species_dict[key] + 6:species_dict[key] + 6 + length])))
        spec_updated = spec.with_columns('sequence', seq, str(length) + '_before', before, str(length) + '_after', after)
        spec_df = spec_updated.to_df()
        if not os.path.isdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species)):
            os.mkdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species))
        spec_df.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + str(species) + "_" + str(pos) + "_raw.fa.csv")

In [29]:
for spec in species:
    getRawPositions(spec, 5)


[60]
Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
2778       | -6.14023 | VT11048|0|MEMB002A|+|284 | 0            | positive | 21             | bcd_FlyReg.fm
2779       | 1.09259  | VT11048|0|MEMB002A|+|284 | 0            | negative | 21             | bcd_FlyReg.fm
2780       | -6.14023 | VT11048|0|MEMB002A|+|284 | 1            | positive | 22             | bcd_FlyReg.fm
2781       | 1.09259  | VT11048|0|MEMB002A|+|284 | 1            | negative | 22             | bcd_FlyReg.fm
2782       | -1.8923  | VT11048|0|MEMB002A|+|284 | 2            | positive | 23             | bcd_FlyReg.fm
2783       | 3.35563  | VT11048|0|MEMB002A|+|284 | 2            | negative | 23             | bcd_FlyReg.fm
2784       | -4.14023 | VT11048|0|MEMB002A|+|284 | 3            | positive | 24             | bcd_FlyReg.fm
2785       | -3.30734 | VT11048|0|MEMB002A|+|284 | 3            | negative | 24             | bcd_FlyReg.fm
2786       | 2.35563  | VT11048

raw,orig_species,score,align_position,strand
39,VT11048|0|MEMB002A|+|284,-5.20163,60,positive
39,VT11048|0|MEMB002B|-|284,-5.20163,60,positive
39,VT11048|0|MEMB002C|-|284,-5.20163,60,positive
35,VT11048|0|MEMB002D|+|269,-4.20163,60,positive
40,VT11048|0|MEMB002E|+|288,0.0078255,60,positive
36,VT11048|0|MEMB002F|+|278,-5.20163,60,positive
39,VT11048|0|MEMB003A|+|278,0.0078255,60,positive
43,VT11048|0|MEMB003B|+|289,-5.20163,60,positive
39,VT11048|0|MEMB003C|-|284,-5.20163,60,positive
35,VT11048|0|MEMB003D|+|274,0.0078255,60,positive


[ 502 1200 1202 1316 1481 1716 1717 2073 2114 2202 2606 2612 2618]
Unnamed: 0 | score     | species                   | raw_position | strand   | align_position | motif
79260      | -5.55526  | VT16679|0|MEMB002A|-|1805 | 0            | positive | 0              | bcd_FlyReg.fm
79261      | -5.55526  | VT16679|0|MEMB002A|-|1805 | 0            | negative | 0              | bcd_FlyReg.fm
79262      | -2.34581  | VT16679|0|MEMB002A|-|1805 | 1            | positive | 1              | bcd_FlyReg.fm
79263      | -3.20163  | VT16679|0|MEMB002A|-|1805 | 1            | negative | 1              | bcd_FlyReg.fm
79264      | 0.0078255 | VT16679|0|MEMB002A|-|1805 | 2            | positive | 2              | bcd_FlyReg.fm
79265      | 0.0078255 | VT16679|0|MEMB002A|-|1805 | 2            | negative | 2              | bcd_FlyReg.fm
79266      | -4.20163  | VT16679|0|MEMB002A|-|1805 | 3            | positive | 3              | bcd_FlyReg.fm
79267      | -2.19381  | VT16679|0|MEMB002A|-|1805 | 3       

raw,orig_species,score,align_position,strand
398,VT16679|0|MEMB002A|-|1805,-6.55526,502,positive
370,VT16679|0|MEMB002B|+|1762,-6.55526,502,positive
398,VT16679|0|MEMB002C|+|1908,-6.55526,502,positive
423,VT16679|0|MEMB002D|-|1976,-6.40326,502,positive
452,VT16679|0|MEMB002E|-|1989,-6.40326,502,positive
401,VT16679|0|MEMB002F|-|1741,-6.40326,502,positive
411,VT16679|0|MEMB003A|-|1764,-6.40326,502,positive
443,VT16679|0|MEMB003B|-|1981,-6.40326,502,positive
398,VT16679|0|MEMB003C|-|1817,-6.55526,502,positive
411,VT16679|0|MEMB003D|+|1762,-6.40326,502,positive


Unnamed: 0 | score     | species                   | raw_position | strand   | align_position | motif
79260      | -5.55526  | VT16679|0|MEMB002A|-|1805 | 0            | positive | 0              | bcd_FlyReg.fm
79261      | -5.55526  | VT16679|0|MEMB002A|-|1805 | 0            | negative | 0              | bcd_FlyReg.fm
79262      | -2.34581  | VT16679|0|MEMB002A|-|1805 | 1            | positive | 1              | bcd_FlyReg.fm
79263      | -3.20163  | VT16679|0|MEMB002A|-|1805 | 1            | negative | 1              | bcd_FlyReg.fm
79264      | 0.0078255 | VT16679|0|MEMB002A|-|1805 | 2            | positive | 2              | bcd_FlyReg.fm
79265      | 0.0078255 | VT16679|0|MEMB002A|-|1805 | 2            | negative | 2              | bcd_FlyReg.fm
79266      | -4.20163  | VT16679|0|MEMB002A|-|1805 | 3            | positive | 3              | bcd_FlyReg.fm
79267      | -2.19381  | VT16679|0|MEMB002A|-|1805 | 3            | negative | 3              | bcd_FlyReg.fm
79268      | -5.55

...

raw,orig_species,score,align_position,strand
2411,VT8646|0|MEMB002A|+|3062,-6.14023,5114,positive
2276,VT8646|0|MEMB002B|+|2935,-5.14023,5114,positive
2318,VT8646|0|MEMB002C|-|2975,-1.8923,5114,positive
2656,VT8646|0|MEMB002D|+|3516,-4.15533,5114,positive
3295,VT8646|0|MEMB002E|-|4143,-7.55526,5114,positive
2953,VT8646|0|MEMB002F|+|3753,-1.8923,5114,positive
1902,VT8646|0|MEMB003A|+|2540,-5.14023,5114,positive
1733,VT8646|0|MEMB003B|+|2425,0.069226,5114,positive
1909,VT8646|0|MEMB003D|+|2591,0.069226,5114,positive
1910,VT8646|0|MEMB003F|+|2578,-5.14023,5114,positive


Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
44626      | -0.9537  | VT8646|0|MEMB002A|+|3062 | 0            | positive | 0              | bcd_FlyReg.fm
44627      | -3.19381 | VT8646|0|MEMB002A|+|3062 | 0            | negative | 0              | bcd_FlyReg.fm
44628      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 1            | positive | 1              | bcd_FlyReg.fm
44629      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 1            | negative | 1              | bcd_FlyReg.fm
44630      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 2            | positive | 2              | bcd_FlyReg.fm
44631      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 2            | negative | 2              | bcd_FlyReg.fm
44632      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 3            | positive | 3              | bcd_FlyReg.fm
44633      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 3            | negative | 3              | bcd_FlyReg.fm
44634      | 3.35563  | VT8646|0|MEM

raw,orig_species,score,align_position,strand
2629,VT8646|0|MEMB002A|+|3062,-3.30734,5543,positive
2464,VT8646|0|MEMB002B|+|2935,-4.20163,5543,positive
2536,VT8646|0|MEMB002C|-|2975,-6.55526,5543,positive
3049,VT8646|0|MEMB002D|+|3516,-3.30734,5543,positive
3657,VT8646|0|MEMB002E|-|4143,-6.55526,5543,positive
3296,VT8646|0|MEMB002F|+|3753,-4.20163,5543,positive
2093,VT8646|0|MEMB003A|+|2540,-4.20163,5543,positive
1948,VT8646|0|MEMB003B|+|2425,-4.20163,5543,positive
2488,VT8646|0|MEMB003C|+|2927,-6.55526,5543,positive
2123,VT8646|0|MEMB003D|+|2591,-4.20163,5543,positive


Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
44626      | -0.9537  | VT8646|0|MEMB002A|+|3062 | 0            | positive | 0              | bcd_FlyReg.fm
44627      | -3.19381 | VT8646|0|MEMB002A|+|3062 | 0            | negative | 0              | bcd_FlyReg.fm
44628      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 1            | positive | 1              | bcd_FlyReg.fm
44629      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 1            | negative | 1              | bcd_FlyReg.fm
44630      | -4.15533 | VT8646|0|MEMB002A|+|3062 | 2            | positive | 2              | bcd_FlyReg.fm
44631      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 2            | negative | 2              | bcd_FlyReg.fm
44632      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 3            | positive | 3              | bcd_FlyReg.fm
44633      | -0.8923  | VT8646|0|MEMB002A|+|3062 | 3            | negative | 3              | bcd_FlyReg.fm
44634      | 3.35563  | VT8646|0|MEM

raw,orig_species,score,align_position,strand
2679,VT8646|0|MEMB002A|+|3062,-4.20163,5599,positive
2514,VT8646|0|MEMB002B|+|2935,-4.20163,5599,positive
2586,VT8646|0|MEMB002C|-|2975,-4.20163,5599,positive
3093,VT8646|0|MEMB002D|+|3516,-4.20163,5599,positive
3702,VT8646|0|MEMB002E|-|4143,-4.20163,5599,positive
3342,VT8646|0|MEMB002F|+|3753,-4.20163,5599,positive
2138,VT8646|0|MEMB003A|+|2540,-4.20163,5599,positive
1997,VT8646|0|MEMB003B|+|2425,-4.20163,5599,positive
2538,VT8646|0|MEMB003C|+|2927,-4.20163,5599,positive
2168,VT8646|0|MEMB003D|+|2591,-4.20163,5599,positive


In [23]:
#-verify whether or not the motif occurs --> the species that are in the original table
#-also include the score
#-from 
#-keep going w positive sequence make length a variable

In [24]:
table = Table.read_table("../data/output/map_motif_bcd_no_threshold/VT11048.fa.csv")
spec = Table().with_column('species', ['MEMB002A', 'MEMB002B', 'MEMB002C', 'MEMB002D', 'MEMB002E', 'MEMB002F', 'MEMB003A', 'MEMB003B', 'MEMB003C', 'MEMB003D', 'MEMB003E', 'MEMB003F', 'MEMB004A', 'MEMB004B', 'MEMB004E', 'MEMB005D', 'MEMB006B', 'MEMB006C', 'MEMB007A', 'MEMB007B', 'MEMB007C', 'MEMB007D', 'MEMB008C','dkik'])
raw_position = []
for pos in a:
    print(table)
    raw = table.where('align_position', are.equal_to(pos)).where('strand', are.equal_to('positive'))
    raw = raw.drop('strand', 'Unnamed: 0', 'motif').relabel('raw_position', str(pos))
    print(raw)
    arr = raw.column(str(pos))
    print(arr)
    spec = spec.with_columns('raw', arr, 'orig_species', raw.column('species'), 'score', raw.column('score'), 'align_position', raw.column('align_position'))
    spec.show()


Unnamed: 0 | score    | species                  | raw_position | strand   | align_position | motif
2778       | -6.14023 | VT11048|0|MEMB002A|+|284 | 0            | positive | 21             | bcd_FlyReg.fm
2779       | 1.09259  | VT11048|0|MEMB002A|+|284 | 0            | negative | 21             | bcd_FlyReg.fm
2780       | -6.14023 | VT11048|0|MEMB002A|+|284 | 1            | positive | 22             | bcd_FlyReg.fm
2781       | 1.09259  | VT11048|0|MEMB002A|+|284 | 1            | negative | 22             | bcd_FlyReg.fm
2782       | -1.8923  | VT11048|0|MEMB002A|+|284 | 2            | positive | 23             | bcd_FlyReg.fm
2783       | 3.35563  | VT11048|0|MEMB002A|+|284 | 2            | negative | 23             | bcd_FlyReg.fm
2784       | -4.14023 | VT11048|0|MEMB002A|+|284 | 3            | positive | 24             | bcd_FlyReg.fm
2785       | -3.30734 | VT11048|0|MEMB002A|+|284 | 3            | negative | 24             | bcd_FlyReg.fm
2786       | 2.35563  | VT11048|0|ME

ValueError: Column length mismatch. New column does not have the same number of rows as table.

In [ ]:
record_dict = SeqIO.to_dict(SeqIO.parse("../data/raw/outlier_rm_with_length_VT11048.fa", "fasta"))
species_dict = dict(zip(spec.column(2), spec.column(1)))
print(species_dict)
seq = []
five_before = []
five_after = []
for key in species_dict:
    sequence = record_dict[key]
    seq.append(seqToString(sequence[species_dict[key]:species_dict[key] + 6]))
    five_before.append(seqToString(sequence[species_dict[key] - 5:species_dict[key]]))
    five_after.append(seqToString(sequence[species_dict[key] + 6:species_dict[key] + 11]))
spec.with_columns('sequence', seq, 'five_before', five_before, 'five_after', five_after)
spec_df = spec.to_df()
print(spec_df)
spec_df.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/11048_raw.fa.csv")


In [30]:
def combineCSV(species):
    final_table = Table()
    raw = np.empty(0)
    orig_species = np.empty(0)
    score = np.empty(0)
    align_position = np.empty(0)
    sequence = np.empty(0)
    before = np.empty(0)
    after = np.empty(0)
    for file in os.listdir("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species)):
        print(species)
        table = Table().read_table("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + file)
        print(file)
        table.show()
        raw = np.append(raw, table.column('raw'))
        orig_species = np.append(orig_species, table.column('orig_species'))
        score = np.append(score, table.column('score'))
        align_position = np.append(align_position, table.column('align_position'))
        sequence = np.append(sequence, table.column('sequence'))
        before = np.append(before, table.column(7))
        after = np.append(after, table.column(8))
    final_table = final_table.with_columns('raw', raw, 'orig_species', orig_species, 'score', score, 'align_position', align_position, 'sequence', sequence, 'before', before, 'after', after)
    final_table.to_csv("../data/output/2.motif_extraction_output/raw_seq_per_align/" + str(species) + "/" + str(species) + "_final_raw.fa.csv")

In [33]:
for s in species:
    combineCSV(s)

11048
11048_60_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,strand,sequence,5_before,5_after
0,39,VT11048|0|MEMB002A|+|284,-5.20163,60,positive,TAAGCC,ATTAA,CACTC
1,39,VT11048|0|MEMB002B|-|284,-5.20163,60,positive,TAAGCC,ATTAA,CACTC
2,39,VT11048|0|MEMB002C|-|284,-5.20163,60,positive,TAAGCC,ATTAA,CACTC
3,35,VT11048|0|MEMB002D|+|269,-4.20163,60,positive,CAATCC,ATTAA,CACTC
4,40,VT11048|0|MEMB002E|+|288,0.0078255,60,positive,CAGCCC,ATTAA,CACTC
5,36,VT11048|0|MEMB002F|+|278,-5.20163,60,positive,CAAGCC,ATTAA,CACTC
6,39,VT11048|0|MEMB003A|+|278,0.0078255,60,positive,CAGCCC,ATTAA,CACTC
7,43,VT11048|0|MEMB003B|+|289,-5.20163,60,positive,CAACCC,ATTAA,CACTC
8,39,VT11048|0|MEMB003C|-|284,-5.20163,60,positive,TAAGCC,ATTAA,CACTC
9,35,VT11048|0|MEMB003D|+|274,0.0078255,60,positive,CAGCCC,ATTAA,CACTC


16679
16679_1316_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,strand,sequence,5_before,5_after
0,769,VT16679|0|MEMB002A|-|1805,-8.55526,1316,positive,GGACAG,CCTAA,AAACC
1,725,VT16679|0|MEMB002B|+|1762,-6.14023,1316,positive,TGAAAA,CCTAA,AAACC
2,775,VT16679|0|MEMB002C|+|1908,-8.55526,1316,positive,GGACAG,CCTAA,AAGCC
3,855,VT16679|0|MEMB002D|-|1976,-6.14023,1316,positive,GGACAA,TTTCA,AACCA
4,871,VT16679|0|MEMB002E|-|1989,-5.14023,1316,positive,GGATAA,TTTCC,AAACA
5,813,VT16679|0|MEMB002F|-|1741,-6.14023,1316,positive,TGACAA,CCTAA,AAACC
6,797,VT16679|0|MEMB003A|-|1764,-6.14023,1316,positive,AAACAA,TCTGA,AAACG
7,868,VT16679|0|MEMB003B|-|1981,-6.14023,1316,positive,GGACAA,TCTCA,AAACA
8,775,VT16679|0|MEMB003C|-|1817,-8.55526,1316,positive,GGACAG,CCTAA,AAACC
9,797,VT16679|0|MEMB003D|+|1762,-6.14023,1316,positive,AAACAA,TCTGA,AAACG


16679
16679_1202_raw.fa.csv


...

Unnamed: 0,raw,orig_species,score,align_position,strand,sequence,5_before,5_after
0,1040,VT8646|0|MEMB002A|+|3062,-6.40326,1688,positive,TTATCT,AAAGT,GTGGA
1,1071,VT8646|0|MEMB002B|+|2935,-6.40326,1688,positive,TAATCT,AAAGT,GTGGA
2,1015,VT8646|0|MEMB002C|-|2975,-6.40326,1688,positive,TTATCT,AAAGT,GAGGA
3,701,VT8646|0|MEMB002D|+|3516,-6.40326,1688,positive,TAATCT,AAAGT,CGGGA
4,699,VT8646|0|MEMB002E|-|4143,-6.40326,1688,positive,TTATCT,AAAGT,CGGGA
5,1341,VT8646|0|MEMB002F|+|3753,-6.40326,1688,positive,TAATCT,AAAGT,GTGGA
6,905,VT8646|0|MEMB003A|+|2540,-6.40326,1688,positive,TAATCT,AAAGT,GGGGA
7,672,VT8646|0|MEMB003B|+|2425,-6.40326,1688,positive,TAATCT,AAAGT,TGGGA
8,1059,VT8646|0|MEMB003C|+|2927,-6.40326,1688,positive,TTATCT,AAAGT,GAGGA
9,895,VT8646|0|MEMB003D|+|2591,-6.40326,1688,positive,TAATCT,AAAGT,GGGGA


8646
8646_1691_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,strand,sequence,5_before,5_after
0,1043,VT8646|0|MEMB002A|+|3062,-0.8923,1691,positive,GGATTA,CCAAA,TTTGT
1,1074,VT8646|0|MEMB002B|+|2935,-5.14023,1691,positive,GGATAA,AAAAA,TTTGT
2,1018,VT8646|0|MEMB002C|-|2975,-0.8923,1691,positive,GGATTA,CCAAA,TTTGA
3,704,VT8646|0|MEMB002D|+|3516,-5.14023,1691,positive,GGATAA,ATAAA,TTGCG
4,702,VT8646|0|MEMB002E|-|4143,-0.8923,1691,positive,GGATTA,ACAAA,TTGCG
5,1344,VT8646|0|MEMB002F|+|3753,-5.14023,1691,positive,GGATAA,ACTAA,CTTGT
6,908,VT8646|0|MEMB003A|+|2540,-5.14023,1691,positive,GGATAA,ACAAA,TTGGG
7,675,VT8646|0|MEMB003B|+|2425,-5.14023,1691,positive,GGATAA,ACAAA,TGCTG
8,1062,VT8646|0|MEMB003C|+|2927,-0.8923,1691,positive,GGATTA,CCAAA,TTTGA
9,898,VT8646|0|MEMB003D|+|2591,-5.14023,1691,positive,GGATAA,ACAAA,TTGGG


8646
8646_5543_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,strand,sequence,5_before,5_after
0,2629,VT8646|0|MEMB002A|+|3062,-3.30734,5543,positive,TCCCTG,ACCGC,CCAGG
1,2464,VT8646|0|MEMB002B|+|2935,-4.20163,5543,positive,TCCCCC,ACCGC,CCAGG
2,2536,VT8646|0|MEMB002C|-|2975,-6.55526,5543,positive,TCCCCG,ACCGC,CCAGG
3,3049,VT8646|0|MEMB002D|+|3516,-3.30734,5543,positive,TTCCTG,ACCGC,TCCAG
4,3657,VT8646|0|MEMB002E|-|4143,-6.55526,5543,positive,TTCCCG,ACCGC,CCGAG
5,3296,VT8646|0|MEMB002F|+|3753,-4.20163,5543,positive,TCCCCC,ACCGC,CCACG
6,2093,VT8646|0|MEMB003A|+|2540,-4.20163,5543,positive,CTCCCC,ACCGC,GCCAG
7,1948,VT8646|0|MEMB003B|+|2425,-4.20163,5543,positive,TTCCCC,ACCGC,CCCGG
8,2488,VT8646|0|MEMB003C|+|2927,-6.55526,5543,positive,TCCCCG,ACCGC,CCAGG
9,2123,VT8646|0|MEMB003D|+|2591,-4.20163,5543,positive,CTCCCC,ACCGC,GCCAG


In [174]:
combineCSV(60074)

60074
60074_44_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,sequence,5_before,5_after
0,16,VT60074|1|MEMB002A|+|926,-3.30734,44,CCATTG,CTTCC,CTCCA
1,15,VT60074|1|MEMB002B|-|943,0.902116,44,TTGCTG,CCAGT,CTCCA
2,16,VT60074|1|MEMB002C|-|914,-3.30734,44,CCATTG,CTTCC,CTCCA
3,16,VT60074|1|MEMB002D|+|920,-2.34581,44,GGGCCG,CCAGA,CTCCA
4,10,VT60074|1|MEMB002E|+|926,-7.55526,44,GGACCG,CCGGA,CTCCA
5,16,VT60074|1|MEMB002F|-|938,-5.14023,44,GGACCA,GGTGC,CGCCG
6,31,VT60074|1|MEMB003A|+|914,-3.30734,44,GCTCCG,CCGGA,CGGTG
7,10,VT60074|1|MEMB003B|-|893,-3.30734,44,GGTCCG,CTTGA,CTCCA
8,16,VT60074|1|MEMB003C|-|926,-3.30734,44,CCATTG,CTTCC,CTCCA
9,31,VT60074|1|MEMB003D|+|926,-3.30734,44,GCTCCG,CCGGA,CGGTG


60074
60074_940_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,sequence,5_before,5_after
0,816,VT60074|1|MEMB002A|+|926,-3.14023,940,GAATGA,AGGCG,GCGCT
1,815,VT60074|1|MEMB002B|-|943,-3.14023,940,GGATGA,AGGCC,GCGCT
2,804,VT60074|1|MEMB002C|-|914,-3.14023,940,GGATGA,AGGCT,GCGCT
3,801,VT60074|1|MEMB002D|+|920,-3.14023,940,GTATGA,AGGCC,GCGCT
4,801,VT60074|1|MEMB002E|+|926,-3.14023,940,GAATGA,AAGCC,ACGCT
5,810,VT60074|1|MEMB002F|-|938,-3.14023,940,GGATGA,AGGCC,GCGCT
6,798,VT60074|1|MEMB003A|+|914,-3.14023,940,GGATGA,AGGCC,GCGCT
7,786,VT60074|1|MEMB003B|-|893,-3.14023,940,GAATGA,AGGCC,GCGCT
8,816,VT60074|1|MEMB003C|-|926,-3.14023,940,GAATGA,AGGCT,GCGCT
9,807,VT60074|1|MEMB003D|+|926,-3.14023,940,GGATGA,AGGCC,GCGCT


60074
60074_575_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,sequence,5_before,5_after
0,505,VT60074|1|MEMB002A|+|926,-0.9537,575,GGTCCC,GAGTT,CTCCT
1,489,VT60074|1|MEMB002B|-|943,-0.9537,575,GGTCCC,GACTG,CTCCT
2,493,VT60074|1|MEMB002C|-|914,-0.9537,575,GGTCCC,GAGTT,CTCCT
3,481,VT60074|1|MEMB002D|+|920,-7.55526,575,GGACCG,GAGTT,CTCCA
4,478,VT60074|1|MEMB002E|+|926,-7.55526,575,GGACCG,GAGTT,CTCCG
5,496,VT60074|1|MEMB002F|-|938,-2.34581,575,TGGCCG,GCGTT,CTCCT
6,478,VT60074|1|MEMB003A|+|914,-7.55526,575,GGACCG,GAATT,CTCCG
7,466,VT60074|1|MEMB003B|-|893,-7.55526,575,GGACCG,GAATT,CTCCA
8,505,VT60074|1|MEMB003C|-|926,-3.30734,575,GGTCCG,GAGTT,CTCCT
9,487,VT60074|1|MEMB003D|+|926,-7.55526,575,GGACCG,GAATT,CTCCG


60074
60074_866_raw.fa.csv


Unnamed: 0,raw,orig_species,score,align_position,sequence,5_before,5_after
0,766,VT60074|1|MEMB002A|+|926,-6.55526,866,TAATCG,GGCTC,TTCGG
1,750,VT60074|1|MEMB002B|-|943,-1.34581,866,TAGTCG,GGCTC,TGCGG
2,754,VT60074|1|MEMB002C|-|914,-6.55526,866,TAATCG,GGCTC,TTCGG
3,748,VT60074|1|MEMB002D|+|920,-1.34581,866,TAGTCG,GGCTC,TGCGG
4,745,VT60074|1|MEMB002E|+|926,1.06923,866,TAGTCA,GGCTC,TGCGG
5,757,VT60074|1|MEMB002F|-|938,-1.34581,866,TAGTCG,GGCTC,TGCGG
6,745,VT60074|1|MEMB003A|+|914,-6.55526,866,TAATCG,GGCTC,TGCGG
7,733,VT60074|1|MEMB003B|-|893,-1.34581,866,TAGTCG,GGCTC,TGCGG
8,766,VT60074|1|MEMB003C|-|926,-6.55526,866,TAATCG,GGCTC,TGCGG
9,754,VT60074|1|MEMB003D|+|926,-6.55526,866,TAATCG,GGCTC,TGCGG
